<a href="https://colab.research.google.com/github/anastasiya-lv/avlvova-project/blob/main/Copy_of_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Загрузка данных, указав delimiter, quotechar и escapechar
try:
    df = pd.read_csv(
        "запчасти авито.csv",
        delimiter=";",  # Изменяем разделитель на точку с запятой
        quotechar='"',
        escapechar="\\",
        on_bad_lines='skip',  # Игнорировать строки с ошибками
        engine='python'
    )
    print("Имена колонок:", df.columns)  # Добавьте эту строку, чтобы увидеть имена колонок
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")
    exit()

# Убедитесь, что 'OEM' и 'Description' есть в df.columns
if 'OEM' not in df.columns or 'Description' not in df.columns:
    print("Отсутствуют необходимые колонки 'OEM' или 'Description' в CSV файле")
    exit()

Имена колонок: Index(['Id', 'AvitoId', 'AvitoStatus', 'AvitoDateEnd', 'ListingFee',
       'Category', 'GoodsType', 'ProductType', 'CarType', 'Title',
       'Description', 'Condition', 'Price', 'ImageUrls', 'Address',
       'ManagerName', 'EMail', 'ContactPhone', 'ContactMethod',
       'AccessoryType', 'Accident', 'activesafety', 'AdType',
       'antitheftsystem', 'Availability', 'BodySparePartType', 'BodyType',
       'Brand', 'BrushBrand', 'BrushLength', 'BrushType', 'climatecontrol',
       'climatecontroloptions', 'Color', 'CompanyName', 'ComplectationId',
       'DifferentWidthTires', 'Doors', 'DriveType', 'drivingassistance',
       'electricdrive', 'EngineSize', 'EngineSparePartType', 'EquipmentType',
       'FuelType', 'generation', 'GenerationId', 'heating',
       'InstallationLocation', 'interior', 'interiorcolor', 'interioroptions',
       'Kilometrage', 'LoadIndex', 'Make', 'Model', 'modification',
       'ModificationId', 'MountingType', 'multimedia', 'OEM', 'OnlyOnAv

In [ ]:
# Функция для добавления текста из колонки OEM в колонку Description при условии, что его нет в Description
def update_description(row):
    oem = str(row['OEM']) if not pd.isna(row['OEM']) else ""
    description = str(row['Description']) if not pd.isna(row['Description']) else ""

    if oem not in description:
        return description + " " + oem
    return description

# Обновление колонки Description
df['Description'] = df.apply(update_description, axis=1)


In [ ]:
# Функция для поиска ссылки на картинку в интернете по названию OEM
def get_image_url(oem):
    search_url = f"https://www.google.com/search?tbm=isch&q={oem}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tag = soup.find("img")
    return img_tag['src'] if img_tag else "No image found"

# Заполнение пустых ячеек в колонке ImageUrls
df['ImageUrls'] = df.apply(lambda row: get_image_url(row['OEM']) if pd.isna(row.get('ImageUrls')) else row['ImageUrls'], axis=1)

# Сохранение обновленного датафрейма обратно в CSV файл
df.to_csv("updated_запчасти_авито.csv", index=False)

print("Обновленный файл сохранен.")

Обновленный файл сохранен.
